<a href="https://colab.research.google.com/github/phossieann/student/blob/main/FOL_representation_of_sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.grammar import FeatureGrammar
from nltk.parse.featurechart import FeatureChart, FeatureChartParser
from nltk.featstruct import Feature
from nltk.sem.logic import Expression, ImpExpression
from typing import Tuple, List, Dict
from nltk.sem.logic import *
from scipy.optimize._lsq.trf_linear import print_header_linear
from nltk.metrics.aline import R
from numpy.core.multiarray import result_type
from scipy.special import exprel

In [2]:
# @title Grammar
simple_grammar = """
    S -> DP[NUM=?n] VP[NUM=?n]
    DP[NUM=?n] -> PropN[NUM=?n] | Quant[NUM=?n] NP[NUM=?n]
    NP[NUM=?n] -> N[NUM=?n] | A N[NUM=?n]
    VP[NUM=?n] -> itV[NUM=?n] | stV[NUM=?n] DP[NUM=?m]

    PropN[NUM=sg] -> 'Vincent' | 'Mia'
    PropN[NUM=pl] -> 'TheJohnsons'
    Quant[NUM=sg] -> 'every' | 'a' | 'no'
    Quant[NUM=pl] -> 'all' | 'some' | 'no'
    A -> 'male' | 'female'
    N[NUM=sg] -> 'boxer'
    N[NUM=pl] -> 'boxers'
    itV[NUM=sg] -> 'walks'
    itV[NUM=pl] -> 'walk'
    stV[NUM=sg] -> 'likes'
    stV[NUM=pl] -> 'like'
"""

In [3]:
# @title Sample sentences
sentences = ['Vincent walks', 'Vincent likes Mia', 'Vincent likes no boxers',
             'TheJohnsons walk', 'TheJohnsons like Vincent', 'TheJohnsons like a boxer',
             'every boxer walks', 'no boxer likes Mia', 'a boxer likes some boxers',
             'no boxers walk', 'some boxers like TheJohnsons', 'all boxers like all boxers']

In [5]:
# @title FOL Representations
gr1 = FeatureGrammar.fromstring(simple_grammar)
parser1 = FeatureChartParser(gr1)
read_expr = Expression.fromstring
sg_rep = {

    'Vincent': ('VINCENT', ('e')),
    'Mia': ('MIA', ('e')),
    'TheJohnsons': ('THEJOHNSONS', ('e')),

    'every': (r'\P.\Q.all x.(P(x) -> Q(x))', (('e', 't'), (('e', 't'), 't'))),
    'a': (r'\P.\Q.exists x.(P(x) ^ Q(x))', (('e', 't'), (('e', 't'), 't'))),
    'no': (r'\P.\Q.-(exists x.(P(x) ^ Q(x)))', (('e', 't'), (('e', 't'), 't'))),
    'all': (r'\P.\Q.all x.(P(x) -> Q(x))', (('e', 't'), (('e', 't'), 't'))),
    'some': (r'\P.\Q.exists x.(P(x) ^ Q(x))', (('e', 't'), (('e', 't'), 't'))),

    'male': (r'\x.MALE(x)', ('e', 't')),
    'female': (r'\x.FEMALE(x)', ('e', 't')),

    'boxer': (r'\x.BOXER(x)', ('e', 't')),
    'boxers': (r'\x.BOXER(x)', ('e', 't')),

    'walks': (r'\x.WALK(x)', ('e', 't')),
    'walk': (r'\x.WALK(x)', ('e', 't')),

    'likes': (r'\y.\x.LIKES(x,y)', ('e', ('e', 't'))),
    'like': (r'\y.\x.LIKES(x,y)', ('e', ('e', 't')))
}

def combine(tree1, tree2):
    return ApplicationExpression(tree1, tree2).simplify()

def get_type(tree, num, rep):
# num determines which interpretation of ambiguous sentences is preferred
    if(len(tree)==2):
        type_1 = get_type(tree[0], num, rep)
        type_2 = get_type(tree[1], num, rep)
# Function Application
        if(type_1 == type_2[0]):
            return type_2[1]
        elif(type_2 == type_1[0]):
            return type_1[1]
# Predicate Modification
        elif((type_1 == ('e', 't')) & (type_2 == ('e', 't'))):
            return ('e', 't')
# Type-Shifting
        elif((type_1 == ('e', ('e', 't'))) & (type_2 == (('e', 't'), 't'))):
            if(num == 0):
                return ('e', 't')
            else:
                return ((('e', 't'), 't'), 't')
        elif((type_2 == ('e', ('e', 't'))) & (type_1 == (('e', 't'), 't'))):
            if(num ==0):
                return ('e', 't')
            else:
                return ((('e', 't'), 't'), 't')
# no rules for this combination of types
        else:
            print('type_error')
    if isinstance(tree[0], nltk.tree.tree.Tree):
        return get_type(tree[0], num, rep)
    return rep[tree[0]][1]

def object_raise(expr, seq):
# seq determines whether there is a single object raise
# or if it is a subject raise followed by an object raise
    combine_1 = combine(expr, read_expr('y'))
    if(seq==0):
        combine_2 = combine(combine_1, read_expr('x'))
    else:
        combine_2 = combine(combine_1, read_expr('P'))
    add_lambda = read_expr(r'\y.' + str(combine_2))
    if(seq==0):
        final = read_expr(r'\Q.\x.(Q(' + str(add_lambda) + '))')
    else:
        final = read_expr(r'\Q.\P.(Q(' + str(add_lambda) + '))')
    return final

def subject_raise(expr):
    combine_1 = combine(expr, read_expr('y'))
    combine_2 = combine(combine_1, read_expr('x'))
    add_lambda = read_expr(r'\x.' + str(combine_2))
    final = read_expr(r'\y.\Q.(Q(' + str(add_lambda) + '))')
    return final

def get_subtree(tree, num, rep):
# num determines which interpretation of ambiguous sentences is preferred
    if(len(tree)==2):
        type_1 = get_type(tree[0], num, rep)
        type_2 = get_type(tree[1], num, rep)
# Function Application
        if(type_1 == type_2[0]):
            return combine(get_subtree(tree[1], num, rep), get_subtree(tree[0], num, rep))
        elif(type_2 == type_1[0]):
            return combine(get_subtree(tree[0], num, rep), get_subtree(tree[1], num, rep))
# Predicate Modification
        elif((type_1 == ('e', 't')) & (type_2 == ('e', 't'))):
            return read_expr(r'\x.(' + str(combine(get_subtree(tree[0], num, rep), read_expr('x'))) + '^' + str(combine(get_subtree(tree[1], num, rep), read_expr('x'))) + ')')
# Type-Shifting for Quantifiers in Object Position
# Assumes object+subject raising only happens under the condition that the V type is (e, (e, t))
        elif((type_1 == ('e', ('e', 't'))) & (type_2 == (('e', 't'), 't'))):
            if(num == 0):
                return combine(object_raise(get_subtree(tree[0], num, rep), 0), get_subtree(tree[1], num, rep))
            else:
                raised = subject_raise(get_subtree(tree[0], num, rep))
                return combine(object_raise(raised, 1), get_subtree(tree[1], num, rep))
        elif((type_2 == ('e', ('e', 't'))) & (type_1 == (('e', 't'), 't'))):
            if(num==0):
                return combine(object_raise(get_subtree(tree[1], num, rep)), 0, get_subtree(tree[0], num, rep))
            else:
                raised = subject_raise(get_subtree(tree[1], num, rep))
                return combine(object_raise(raised, 1), get_subtree(tree[0], num, rep))
# For statements where no compositional rule may be applied
        else:
            print('tree_error')
    if isinstance(tree[0], nltk.tree.tree.Tree):
        return get_subtree(tree[0], num, rep)
    return read_expr(rep[tree[0]][0])

# Testing sentences from sample
for sent in sentences:
    for tree in parser1.parse(sent.split()):
          print(sent)
          print(get_subtree(tree, 0, sg_rep))
          print()

# Testing sentences with multiple interpretations
test = 'some boxers like all boxers'
print('TEST:')
print(test)
for tree in parser1.parse(test.split()):
    print('\nINTERPRETATION 1:')
    print(get_subtree(tree, 0, sg_rep))
    print('\nINTERPRETATION 2:')
    print(get_subtree(tree, 1, sg_rep))

test = 'all boxers like some boxers'
print('\nTEST:')
print(test)
for tree in parser1.parse(test.split()):
    print('\nINTERPRETATION 1:')
    print(get_subtree(tree, 0, sg_rep))
    print('\nINTERPRETATION 2:')
    print(get_subtree(tree, 1, sg_rep))

Vincent walks
WALK(VINCENT)

Vincent likes Mia
LIKES(VINCENT,MIA)

Vincent likes no boxers
-exists z1.(BOXER(z1) & LIKES(VINCENT,z1))

TheJohnsons walk
WALK(THEJOHNSONS)

TheJohnsons like Vincent
LIKES(THEJOHNSONS,VINCENT)

TheJohnsons like a boxer
exists z2.(BOXER(z2) & LIKES(THEJOHNSONS,z2))

every boxer walks
all x.(BOXER(x) -> WALK(x))

no boxer likes Mia
-exists x.(BOXER(x) & LIKES(x,MIA))

a boxer likes some boxers
exists x.(BOXER(x) & exists z3.(BOXER(z3) & LIKES(x,z3)))

no boxers walk
-exists x.(BOXER(x) & WALK(x))

some boxers like TheJohnsons
exists x.(BOXER(x) & LIKES(x,THEJOHNSONS))

all boxers like all boxers
all x.(BOXER(x) -> all z4.(BOXER(z4) -> LIKES(x,z4)))

TEST:
some boxers like all boxers

INTERPRETATION 1:
exists x.(BOXER(x) & all z5.(BOXER(z5) -> LIKES(x,z5)))

INTERPRETATION 2:
all x.(BOXER(x) -> exists z7.(BOXER(z7) & LIKES(z7,x)))

TEST:
all boxers like some boxers

INTERPRETATION 1:
all x.(BOXER(x) -> exists z8.(BOXER(z8) & LIKES(x,z8)))

INTERPRETATION 2:
e

In [7]:
# @title Pretty Print
def prettyprint(sent, num, rep):
    # format of nodes/leafs
    """
    phrase
    part of speech
    type
    lambda representation
    """
    # returns type as readable text
    def typeToText(type):
        if(len(type) == 2):
            text = "(" + typeToText(type[0]) + " -> " + typeToText(type[1]) + ")"
            return text
        else:
            return str(type[0])

    # returns multi-line strings next to one another, distance
    def sbs(s1, s2):
        # returns list of each line in a string
        def lineList(s):
            # slist is the list of each line in the string
            slist = []
            start = 0
            longest = 0
            # cuts string into pieces around \n
            for i in range(len(s)):
                if(s[i] == '\n'):
                    slist.append(s[start:i])
                    if ((i-start) > longest):
                      longest = (i-start)
                    start = i + 1
            slist.append(s[start:])
            # makes all pieces of slist the same length as the longest
            for i in range(len(slist)):
                for j in range(longest - (len(slist[i]))):
                    slist[i] = slist[i] + " "
            # returns slist
            return slist
        list1 = lineList(s1)
        list2 = lineList(s2)
        toReturn = ""
        numtabs = 3
        tabspace = 5
        space = ""
        # creates space of length determined by numtabs
        for i in range(numtabs*tabspace):
            space = space + " "
        # if the strings have the same number of lines
        if (len(list1) == len(list2)):
            for i in range(len(list1)):
                toReturn = toReturn + list1[i] + space + list2[i] + "\n"
            distance = len(list1[0]) + len(space)
            return (toReturn, distance)
        # if s1 has more lines than s2
        elif (len(list1) > len(list2)):
            for i in range(len(list1)):
                if (i < len(list2)):
                    toReturn = toReturn + list1[i] + space + list2[i] + "\n"
                else:
                    toReturn = toReturn + list1[i] + "\n"
            distance = len(list1[0]) + len(space)
            return (toReturn, distance)
        # if s2 has more lines than s1
        else:
            # finds the common length in list1 and adds the space before printing later parts of list2
            firstSpace = ""
            for i in range(len(list1[0])):
                firstSpace = firstSpace + " "
            for i in range(len(list2)):
                if (i < len(list1)):
                    toReturn = toReturn + list1[i] + space + list2[i] + "\n"
                else:
                    toReturn = toReturn + firstSpace + space + list2[i] + "\n"
            distance = len(list1[0]) + len(space)
            return (toReturn, distance)
    # recursive function that gives daughter trees/leaves of node
    def get_text(tree, num):
        # tree lines
        vert = "|\n|\n|\n"
        horiz = "-"
        # if tree has two children
        if(len(tree) == 2):
            s1 = vert + get_text(tree[0], num)
            s2 = vert+ get_text(tree[1], num)
            for i in range(sbs(s1, s2)[1]):
                horiz = horiz + "-"
            simpletype = typeToText(get_type(tree, num, rep))
            repr = str(get_subtree(tree, num, rep))
            text = simpletype + "\n" + repr + "\n"
            return (text + vert + horiz + "\n" + sbs(s1, s2)[0])
        # if tree has one child
        elif(isinstance(tree[0], nltk.tree.tree.Tree)):
            return (get_text(tree[0], num))
        # if tree is leaf
        else:
            word = str(tree[0])
            pos = str(tree.label())
            simpletype = typeToText(get_type(tree, num, rep))
            repr = str(get_subtree(tree, num, rep))
            text = word + "\n" + pos + "\n" + simpletype + "\n" + repr + "\n"
            return text

    for tree in parser1.parse(sent.split()):
        print(sent)
        print(get_text(tree, num))

# tests
for sent in sentences:
    prettyprint(sent, 0, sg_rep)

prettyprint("all boxers like some boxers", 0, sg_rep)
prettyprint("all boxers like some boxers", 1, sg_rep)

Vincent walks
t
WALK(VINCENT)
|
|
|
------------------------------------
|                                  |                 
|                                  |                 
|                                  |                 
Vincent                            walks             
[ *type* = 'PropN' ]               [ *type* = 'itV' ]
[ NUM    = 'sg'    ]               [ NUM    = 'sg'  ]
e                                  (e -> t)          
VINCENT                            \x.WALK(x)        
                                                     

Vincent likes Mia
t
LIKES(VINCENT,MIA)
|
|
|
------------------------------------
|                                  |                                                    
|                                  |                                                    
|                                  |                                                    
Vincent                            (e -> t)                                             
[ 